In [7]:
import spacy
import re

In [8]:
import re

def extraer_metadatos_gaceta(texto_pdf):
    """Extrae metadatos de una gaceta del Congreso en formato texto."""

    metadatos = {}

    # --- 1. Fecha ---
    patron_fecha = r"(?i)(lunes|martes|miércoles|jueves|viernes|sábado|domingo),\s*(\d{1,2})\s+de\s+(enero|febrero|marzo|abril|mayo|junio|julio|agosto|septiembre|octubre|noviembre|diciembre)\s+de\s+(\d{4})"
    coincidencia_fecha = re.search(patron_fecha, texto_pdf)
    if coincidencia_fecha:
        meses = {"enero": 1, "febrero": 2, "marzo": 3, "abril": 4, "mayo": 5, "junio": 6,
                 "julio": 7, "agosto": 8, "septiembre": 9, "octubre": 10, "noviembre": 11, "diciembre": 12}
        dia_mes = int(coincidencia_fecha.group(2))
        mes_numero = meses[coincidencia_fecha.group(3).lower()]
        anio = int(coincidencia_fecha.group(4))
        metadatos["fecha"] = f"{dia_mes:02d}/{mes_numero:02d}/{anio}"

    # --- 2. Directores ---
    patron_directores = r"(?i)DIRECTORES:\s*(.*?)\s*(?:SECRETARIO|RAMA)"
    coincidencia_directores = re.search(patron_directores, texto_pdf)
    if coincidencia_directores:
        metadatos["directores"] = coincidencia_directores.group(1).strip()

    # --- 3. Edición de n páginas ---
    patron_paginas = r"(?i)EDICI[ÓO]N\s+DE\s+(\d+)\s+P[ÁA]GINAS"
    coincidencia_paginas = re.search(patron_paginas, texto_pdf)
    if coincidencia_paginas:
        metadatos["num_paginas"] = int(coincidencia_paginas.group(1))

    # --- 4. Año (Números Romanos) ---
    patron_anio = r"AÑO\s+(M{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3}))"
    coincidencia_anio = re.search(patron_anio, texto_pdf)
    if coincidencia_anio:
        romano = coincidencia_anio.group(1)
        metadatos["anio_romano"] = romano
        metadatos["anio"] = romano_a_entero(romano)  # Conversión a entero

    # --- 5. Nombre Principal ---
    patron_nombre = r"(?i)GACETA\s+DEL\s+CONGRESO"
    if re.search(patron_nombre, texto_pdf):
        metadatos["nombre"] = "Gaceta del Congreso"

    # --- 6. Subtítulo ---
    patron_subtitulo = r"(?i)GACETA\s+DEL\s+CONGRESO\s*\n*(.*?)\s*(?:AÑO|DIRECTORES)"
    coincidencia_subtitulo = re.search(patron_subtitulo, texto_pdf)
    if coincidencia_subtitulo:
        metadatos["subtitulo"] = coincidencia_subtitulo.group(1).strip()

     # --- 7. Artículo, Ley ---
    patron_articulo =  r"(?i)ART[ÍI]CULO\s+\d+(?:\s*(?:[º°ª])|\s*[Nn][°º])[\s.,:;]*LEY\s+\d+(?:\s*(?:[º°ª])|\s*[Nn][°º])[\s.,:;]*DE[\s.,:;]*\d+"
    if re.search(patron_articulo, texto_pdf):
        metadatos["articulo_ley"] = True


    # --- 8. ISSN ---
    patron_issn = r"(?i)ISSN\s+(\d{4}-\d{3}[\dX])"
    coincidencia_issn = re.search(patron_issn, texto_pdf)
    if coincidencia_issn:
        metadatos["issn"] = coincidencia_issn.group(1)

    # --- 9. Rama Legislativa ---
    patron_rama = r"(?i)RAMA\s+LEGISLATIVA\s+DEL\s+PODER\s+P[ÚU]BLICO"
    if re.search(patron_rama, texto_pdf):
        metadatos["rama"] = "Rama Legislativa del Poder Público"


    # --- 10. Cámara o Senado ---
    patron_camara = r"(?i)(C[ÁA]MARA\s+DE\s+REPRESENTANTES|SENADO\s+DE\s+LA\s+REP[ÚU]BLICA)"
    coincidencia_camara = re.search(patron_camara, texto_pdf)
    if coincidencia_camara:
        metadatos["camara"] = coincidencia_camara.group(1)


    # --- 11. Tipo de Documento ---  CORRECCIÓN AQUÍ
    patron_tipo = r"^(Ponencia|Acta|Proyecto\s+de\s+Ley|Informe|Resolución|Concepto|Proposición|Constancia|Objeciones|Textos|Pliego)(.*)"
    for linea in texto_pdf.splitlines():  # Procesa línea por línea
        coincidencia_tipo = re.search(patron_tipo, linea, re.IGNORECASE)  # Usar re.IGNORECASE
        if coincidencia_tipo:
            metadatos["tipo_documento"] = coincidencia_tipo.group(1).strip()
            metadatos["descripcion"] = coincidencia_tipo.group(2).strip() #descripcion
            break  # Sale del bucle después de encontrar la primera coincidencia.

    return metadatos


In [9]:
def romano_a_entero(romano):
    """Convierte un número romano a entero."""
    valores = {'M': 1000, 'CM': 900, 'D': 500, 'CD': 400, 'C': 100, 'XC': 90,
               'L': 50, 'XL': 40, 'X': 10, 'IX': 9, 'V': 5, 'IV': 4, 'I': 1}
    entero = 0
    i = 0
    while i < len(romano):
        if i + 1 < len(romano) and romano[i:i+2] in valores:
            entero += valores[romano[i:i+2]]
            i += 2
        else:
            entero += valores[romano[i]]
            i += 1
    return entero

In [10]:
metadatos = extraer_metadatos_gaceta()
print(metadatos)
# Salida esperada (aproximada, puede variar ligeramente según la limpieza exacta):
# {'fecha': '09/02/2023', 'directores': 'GREGORIO ELJACH PACHECO\nSECRETARIO GENERAL DEL SENADO\nwww.secretariasenado.gov.co', 'num_paginas': 22, 'anio_romano': 'XXXII', 'anio': 32, 'nombre': 'Gaceta del Congreso', 'subtitulo': 'CONCEPTOS JURÍDICOS', 'issn': '0123-9066', 'rama': 'Rama Legislativa del Poder Público', 'camara': 'SENADO DE LA REPÚBLICA', 'tipo_documento': 'CONCEPTOS'}

TypeError: extraer_metadatos_gaceta() missing 1 required positional argument: 'texto_pdf'